In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [2]:
base_url = 'https://www.chillfactore.com/plan-your-visit/events'

headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

## raw scraped data

In [3]:
response = requests.get(
    base_url,
    # headers=headers,
    timeout=5
)

soup = BeautifulSoup(response.content, 'html.parser')

Looking for a match like:

```
<figure data-day="22" data-month="Mar">
<div>
<a href="/lift-passes/terrain-challenges"><img 193"="" 308"="" alt="Moguls" height="&lt;img src=" loading="lazy" src="/perch/resources/cf-website-events9-w330h250.jpg" width="&lt;img src="/></a>
</div>
</figure>
```

In [4]:
"""
figures = soup.select('figure')

print(
    figures[16]
)

figures[16].find('a', text='Moguls')
"""

"\nfigures = soup.select('figure')\n\nprint(\n    figures[16]\n)\n\nfigures[16].find('a', text='Moguls')\n"

In [5]:
dates_with_moguls = list()

day_match_re = r'(?<=data-day=")(.*)(?=" data-month)'
month_match_re = r'(?<=data-month=")(.*)(?=")'

year = '2024'

for i in soup.find_all(['figure']):
    if 'alt="Moguls"' in str(i):
        # strip out date info
        _i = str(i)
        day = re.search(day_match_re, _i).group(0)
        month = re.search(month_match_re, _i).group(0)
        
        # format
        date = '-'.join([year, month, day])
        date = dt.datetime.strptime(date, '%Y-%b-%d').date()
        
        dates_with_moguls.append(date)

dates_with_moguls

[datetime.date(2024, 3, 21),
 datetime.date(2024, 3, 22),
 datetime.date(2024, 4, 11),
 datetime.date(2024, 4, 12),
 datetime.date(2024, 4, 13),
 datetime.date(2024, 5, 9),
 datetime.date(2024, 5, 10),
 datetime.date(2024, 5, 11),
 datetime.date(2024, 6, 6),
 datetime.date(2024, 6, 7),
 datetime.date(2024, 6, 8),
 datetime.date(2024, 7, 4),
 datetime.date(2024, 7, 5),
 datetime.date(2024, 7, 6),
 datetime.date(2024, 8, 1),
 datetime.date(2024, 8, 2),
 datetime.date(2024, 8, 3),
 datetime.date(2024, 8, 29),
 datetime.date(2024, 8, 30),
 datetime.date(2024, 8, 31)]

## google events

In [7]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle

In [11]:
dt.datetime(
    dates_with_moguls[0].year,
    dates_with_moguls[0].month,
    dates_with_moguls[0].day,
    10
).isoformat()

'2024-03-21T10:00:00'